# Data Cleaning / Wrangling w/ Pandas

About pandas:

pandas is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool,
built on top of the Python programming language.

In [ ]:
# First step, import the pandas package

import pandas as pd

In [ ]:
# Next step, load the file that we want to clean

file = pd.read_excel('/Users/daniel.geanon/OneDrive - Karolinska Institutet/Mac/Desktop/211126_CIMOverflow.xlsx')

In [ ]:
# Display file

file

In [ ]:
# First step, let's read columns and check the names

file.columns.tolist()

In [ ]:
# There are many typos in the column names, let's correct those

example_string = 'r√∂kare'
example_string = example_string.replace('√∂','ö')
example_string

In [ ]:
# There's a number of instances where swedish was misread into english
# Let's change all of those characters back in a for loop using str.replace()

new_col_name_list = []

for column in file.columns.tolist():
    new_col = column.replace('√∂','ö').replace('√§','ä').replace('√•','å')
    new_col_name_list.append(new_col)
    
new_col_name_list

In [ ]:
# Column names are now clean, so let's rename the columns with our new list

file.columns = new_col_name_list
file.columns

In [ ]:
# Showing file

file

In [ ]:
# Let's loop through every row and do the same sort of cleaning
for value in list(range(0,392)):
    for column in file.columns.tolist():
        cell = file.loc[value,column]
        if type(cell) == str:
            cell = cell.replace('√∂','ö').replace('√§','ä').replace('√•','å')
            file.loc[value,column] = cell
        
file['Smoking (nuvarande/tidigare/ej rökare)'].value_counts()

In [ ]:
# Let's explore the "Include?" Column

file['Include?'].value_counts()

In [ ]:
# 0 must equal exclude, so let's drop those from our dataframe
# First, let's locate all rows where Include? == 0, get the indices for those rows, and assign them to a new list

drop_list = file.loc[file['Include?'] == 0].index.tolist()
drop_list

In [ ]:
# Now, let's drop those rows with those indices

file = file.drop(drop_list)

file

In [ ]:
# Let's explore more columns and see what further cleaning we can do

file.columns.tolist()

In [ ]:
# Cleaning one more column
file = file.rename(columns = {'Date disease onsent':'Date disease onset'})
file.columns.tolist()

In [ ]:
# There are lots of relevant dates, and we want to calculate day intervals
# 1) Date biobank sample
# 2) Date disease onset
# 3) Date hospital admission
# 4) Date positive PCR test

In [ ]:
# Let's create new columns with day ranges for each of those conditions

# First, we need to check the datetime format of these dates

file['Date for biobank sample']

In [ ]:
# Example (locating one date in the dataframe)

d1 = file.loc[0,'Date disease onset']
d1.strftime("%Y-%m-%d")

In [ ]:
d2 = file.loc[0,'Date for biobank sample']
d2.strftime("%Y-%m-%d")

In [ ]:
# Define days_between function

from datetime import datetime

def days_between(d1, d2):
    d1 = datetime.strptime(d1.strftime("%Y-%m-%d"), "%Y-%m-%d")
    d2 = datetime.strptime(d2.strftime("%Y-%m-%d"), "%Y-%m-%d")
    return (d2 - d1).days

In [ ]:
days_between(d1,d2)

In [ ]:
# Let's use this function for every value in our dataframe
# First we will reset dataframe indices so that they are in a consecutive integer list

file = file.reset_index()
file

In [ ]:
file = file.rename(columns = {'index':'original index'})
file

In [ ]:
file['Date disease onset'].unique().tolist()

In [ ]:
# We have some non date values ('na' & 0), so we'll have to take care of that in our function)

file['Days since symptom onset'] = ''

for value in list(range(0,387)):
    d1 = file.loc[value,'Date disease onset']
    d2 = file.loc[value, 'Date for biobank sample']
    if d1 == 'na' or d2 == 'na' or d1 == 0 or d2 == 0 :
        file.loc[value, 'Days since symptom onset'] = 'NaN'
    else:
        days = days_between(d1,d2)
        file.loc[value,'Days since symptom onset'] = days
    
file

In [ ]:
file['Days since symptom onset'].tolist()

In [ ]:
file['Date hospital admission'].unique().tolist()

In [ ]:
# 2) Date since hospital admission

file['Days since hospital admission'] = ''

for value in list(range(0,387)):
    d1 = file.loc[value,'Date hospital admission']
    d2 = file.loc[value, 'Date for biobank sample']
    if d1 == 'na' or d2 == 'na' or d1 == 0 or d2 == 0 :
        file.loc[value, 'Days since hospital admission'] = 'NaN'
    else:
        days = days_between(d1,d2)
        file.loc[value,'Days since hospital admission'] = days
    
file

In [ ]:
file['Date positive PCR test'].unique().tolist()

In [ ]:
# 2) Date since positive pcr test

file['Days since positive PCR test'] = ''

for value in list(range(0,387)):
    d1 = file.loc[value,'Date positive PCR test']
    d2 = file.loc[value, 'Date for biobank sample']
    if d1 == 0 or d2 == 0 :
        file.loc[value, 'Days since positive PCR test'] = 'NaN'
    else:
        days = days_between(d1,d2)
        file.loc[value,'Days since positive PCR test'] = days
    
file

In [ ]:
# Relevant day ranges are added, let's explore other columns that we can clean
file.columns.tolist()

In [ ]:
# Let's explore some of the variables in our cohort further
file['Age'].describe()

In [ ]:
file['Sex'].value_counts()

In [ ]:
file['BMI final'].describe()

In [ ]:
file['Smoking (nuvarande/tidigare/ej rökare)'].value_counts()

In [ ]:
file['CCI (Charlson comorbidity index)'].describe()

In [ ]:
# From here, I can come up with a few challenges

In [ ]:
# Patients older than 50

file.loc[file['Age'] > 50]

In [ ]:
# What sort of treatment are they on
file.loc[file['Age'] > 50]['Cytokinblockad före biobanksprov1'].value_counts()

In [ ]:
# Mean neutrophil count for biobank 1

file.loc[file['Biobank Sample'] == 1]['Neutrophil ct +/- 24 hrs'].mean()

In [ ]:
# Compared to Later in infection
file.loc[file['Biobank Sample'] == 3]['Neutrophil ct +/- 24 hrs'].mean()